In [9]:
from obsidianScripts.main import main
import json
from gptStuff.masterGPT import GPTFlow

json_data = None
with open('obsidianScripts/testJSONS/test1.json', 'r') as file:
    json_data = file.read()
    file.close()
json_data=json.loads(json_data)


main(json_data=json_data,debug=True)

safemode is on
this will not write to the files
debug mode is on
{'type': 'memory', 'person': 'Harshini', 'location': {'0': 'Important Details', '1': 'Relationship'}, 'item': 'Roommates with Ananya'}
Harshini Kundurthi
attribute classifier dict_keys(['Likes and Dislikes', 'Important Details', 'Gift Ideas', 'Birthday', 'Dietary Restrictions', 'pronouns', 'tags'])
first location Important Details
['1']
['1']
**Relationship:** 
*Elaborate if you want!*
here
file written


In [10]:
# get the token from the json
with open('token.json', 'r') as file:
    token = file.read()
    file.close()
token = json.loads(token)
token = token['token']


7266721080:AAGBaAytkmacRQgLs8DWp271Fh3wqRSDL6U


In [11]:
import requests

def set_webhook(bot_token, webhook_url):
    url = f'https://api.telegram.org/bot{bot_token}/setWebhook'
    payload = {
        'url': webhook_url
    }
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        print('Webhook set successfully!')
    else:
        print('Failed to set webhook.')
        print('Error:', response.text)

# Replace <YOUR_BOT_TOKEN> with your actual bot token
webhook_url = 'a812-2600-1700-2b1-b600-183f-f396-74e3-9819.ngrok-free.app'+'/telegram-webhook'

set_webhook(token, webhook_url)

Webhook set successfully!


In [12]:
from flask import Flask, request
import subprocess

import requests
app = Flask(__name__)


@app.route('/telegram-webhook', methods=['POST'])
def webhook():
    # Get the input data from the request
    input_data = request.json

    # Extract the relevant information from the input data
    try:
        message_text = input_data['message']['text']
        chat_id = input_data['message']['chat']['id']
    except:
        print('Invalid input data:', input_data)
        return 'OK'

    # Run the Python script with the input data
    result = run_python_script(message_text)

    # Send the result back to Telegram
    send_message(chat_id, result)

    return 'OK'

def run_python_script(input_text):
    # Run the Python script as a subprocess and capture the output
    try:
        response = GPTFlow(input_text)
    except Exception as e:
        response = f'An error occurred: {e}'

    return response

def send_message(chat_id, text):
    # Send a message back to Telegram using the sendMessage method
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': text
    }
    requests.post(url, json=payload)

if __name__ == '__main__':
    print('Starting the Flask app')
    print('token:',token)
    app.run(port=3000)

Starting the Flask app
token: 7266721080:AAGBaAytkmacRQgLs8DWp271Fh3wqRSDL6U
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


inside parent What are desserts that both Areeba and Abeera can eat
{'sister': {'details': 'Retrieve dietary preferences, allergies, and favorite foods for Areeba and Abeera'}, 'brother': {'context': 'Based on the dietary preferences, allergies, and favorite foods provided for Areeba and Abeera, suggest desserts that both of them can eat.'}}
sister result {'type': 'list', 'content': [{'type': 'retrieval', 'person': 'Areeba', 'location': {'0': 'Dietary Restrictions'}}, {'type': 'retrieval', 'person': 'Abeera', 'location': {'0': 'Dietary Restrictions'}}, {'type': 'retrieval', 'person': 'Areeba', 'location': {'0': 'Likes and Dislikes', '1': 'Likes', '2': 'Food'}}, {'type': 'retrieval', 'person': 'Abeera', 'location': {'0': 'Likes and Dislikes', '1': 'Likes', '2': 'Food'}}]}
safemode is on
this will not write to the files
safemode is on
this will not write to the files
safemode is on
this will not write to the files
[]
safemode is on
this will not write to the files
the level Food was not 

127.0.0.1 - - [19/Jun/2024 23:07:22] "POST /telegram-webhook HTTP/1.1" 200 -


inside parent What are desserts that both Areeba and Abeera can eat
{'sister': {'details': 'Retrieve dietary preferences and allergies for Areeba and Abeera'}}
sister result {'type': 'list', 'content': [{'type': 'retrieval', 'person': 'Areeba', 'location': {'0': 'Dietary Restrictions'}}, {'type': 'retrieval', 'person': 'Abeera', 'location': {'0': 'Dietary Restrictions'}}]}
safemode is on
this will not write to the files
safemode is on
this will not write to the files

<Areeba>
Dietary Restrictions:Dietary Restrictions:
  - halal
  - fermented foods
  - yogurt
  - cheese
  - dairy free
  - tomatoes
  - spinach
  - citrus
  - peanut free
  - shellfish
  - sugar
  - caffiene
  - processed foods
  - High histamine
  - Avocado
  - Eggs
  - Nuts
</Areeba>

<Abeera>
Dietary Restrictions:Dietary Restrictions:
  - Lactose Intolerant
  - Gluten Free


127.0.0.1 - - [19/Jun/2024 23:07:28] "POST /telegram-webhook HTTP/1.1" 200 -


inside parent What are more desserts that both Areeba and Abeera can eat, give me some interesting ones other than cookies
{'sister': {'details': 'What are the dietary preferences, allergies, and favorite foods for Areeba and Abeera'}, 'brother': {'context': 'Come up with dessert ideas based on the dietary preferences, allergies, and favorite foods for Areeba and Abeera provided by the sister.'}}
sister result {'type': 'list', 'content': [{'type': 'retrieval', 'person': 'Abeera', 'location': {'0': 'Dietary Restrictions'}}, {'type': 'retrieval', 'person': 'Areeba', 'location': {'0': 'Dietary Restrictions'}}, {'type': 'retrieval', 'person': 'Abeera', 'location': {'0': 'Likes and Dislikes', '1': 'Likes', '2': 'Food'}}, {'type': 'retrieval', 'person': 'Areeba', 'location': {'0': 'Likes and Dislikes', '1': 'Likes', '2': 'Food'}}]}
safemode is on
this will not write to the files
safemode is on
this will not write to the files
safemode is on
this will not write to the files
the level Food was

127.0.0.1 - - [19/Jun/2024 23:07:58] "POST /telegram-webhook HTTP/1.1" 200 -
